# Herramientas de apoyo para el análisis y procesado de datos del secuenciador de ADN MinION

**Autor:** Orlandy Ariel Sánchez Acosta.

**Correo:** alu0100773408@ull.edu.es

**Asignatura:** Trabajo de Fin de Grado.

**Este cuaderno está pensado para realizar búsquedas en BLAST, de manera que se pueda comprobar si existe similitud entre una secuencia que se le proporcione y otra/as de en la base de datos de BLAST.**

In [1]:
FICHERO_ENTRADA = "Salidas/Coincidencia/Lecturas_malas.fastq"
FORMATO_ENTRADA = "fastq"
FICHERO_SALIDA = "Salidas/Coincidencia/Lecturas_malas.fasta"
FORMATO_SALIDA = "fasta"
#####
PROGRAMA = "blastn"
BASE_DE_DATOS = "nr"

# Comprobación con BLAST

Ahora se pasará a comprobar que, de entre las secuencias que coinciden como error tanto en un filtrado como en el otro, la base de Datos [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi). Puesto que la api con la que cuenta Biopython no funciona como debería, se pasó a realizar una "propia" con la cual utilizando [Web Scraping](https://es.wikipedia.org/wiki/Web_scraping) y en combinación con la propia API web de BLAST, haciendo una combinación de estas se pudo realizar las peticiones a BLAST.

In [2]:
from modolos import putblast as put
from modolos import getblast as get
from Bio import SeqIO
import time
from modolos import Utilidades as utile

# Librerías
En caso de dudas o cualquier consulta sobre las librerías usadas, consultar los siguientes enlaces.
* [Biopython](http://biopython.org/wiki/Documentation)
* [Iteratools](https://docs.python.org/2/library/itertools.html)
* [Peticiones BLAST](https://github.com/OrlandyAriel/BusquedaBLAST)
* [Time](https://docs.python.org/2/library/time.html)

In [3]:
utilidad = utile.Utilidades()
peticionPut = put.PeticionPUTBlast()
peticionGet = get.PeticionGETBlast()

dic_rids = {} # diccionario donde se guardarán los rid's obtenidos de las busquedas en blast
resultados = []

In [4]:
utilidad.convertidor_formatos(FICHERO_ENTRADA,FORMATO_ENTRADA,FICHERO_SALIDA,FORMATO_SALIDA)

In [5]:
#Lectura del fichero .fasta
fich_fasta  = [rec for rec in SeqIO.parse(FICHERO_SALIDA, FORMATO_SALIDA)]
print(len(fich_fasta))

156


In [6]:
for i, sec in enumerate(fich_fasta):
    sec_formateada = sec.seq
    if(len(sec_formateada) > 8000):
        sec_formateada = sec_formateada[0:8000]
    
    
    peticionPut.construir_peticion(sec_formateada,
                                  BASE_DE_DATOS,
                                  PROGRAMA)
    peticionPut.realizar_peticion()
    dic_rids[peticionPut.rid] = sec_formateada
    time.sleep(11)


> **Nota:** Tras realizar distintas pruebas se pudo llegar a la conclusión de que, para realizar las consultas, la secuencia no debe sobrepasar los 8100 caracteres, para no llegar hasta dicho extremo las secuencias la limitaremos a 8000 para evitar posibles errores. Además,g se añadió un tiempo de espera de 11 segundos para evitar que el servidor de BLAST detecte las peticiones como ataque y bloquee el acceso a esta desde la IP donde se está realizando las peticiones.

In [7]:
print(len(dic_rids))

156


In [8]:
resultados = []
for rid, sec in dic_rids.items():
    peticionGet.construir_peticion(rid)
    resultados.append(peticionGet.realizar_peticion())

In [ ]:
print("Errores detectados que NO encontró coincidencias en BLAST: ",resultados.count(False))
print("Errores detectados que SI encontró coincidencias en BLAST: ",resultados.count(True))
porcentaje = '{0:.2f}'.format((resultados.count(False) / len(resultados)) * 100)
print("Porcentaje de acierto:", porcentaje)
p = 100- float(porcentaje)
print("Porcentaje de errores:",p)